In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os

# Lendo o arquivo e criando o dataset

In [2]:
BASEFOLDER = r'C:\Users\gabri\OneDrive\Documentos\Projetos\HData\Datalake\{stage}\{folder}'
folder_path = BASEFOLDER.format(stage='bronze', folder='dados_recebidos')
dataset_lancamento = pd.DataFrame()
dataset_saldo = pd.DataFrame()
dataset_cdc = pd.DataFrame()
dataset_contas = pd.DataFrame()

for file in os.listdir(folder_path):
    if file.endswith('xlsx'):
        # Carrega o arquivo Excel
        xls = pd.ExcelFile(os.path.join(folder_path,file))

        # Criando os datasets
        dataset_lancamento = pd.read_excel(os.path.join(folder_path,file), sheet_name=xls.sheet_names[0])
        dataset_saldo = pd.read_excel(os.path.join(folder_path,file), sheet_name=xls.sheet_names[1])
        dataset_cdc = pd.read_excel(os.path.join(folder_path,file), sheet_name=xls.sheet_names[2])
        dataset_contas = pd.read_excel(os.path.join(folder_path,file), sheet_name=xls.sheet_names[3])


# Tratamentos

## Tratando saldo

In [3]:
rename_saldo = {
    'CONTA BANCARIA (REF)': 'ds_conta_bancaria',
    'AGENCIA': 'cd_agencia',
    'CONTA': 'cd_conta_bancaria',
    'DATA SALDO INICIAL': 'dt_saldo_inicial',
    'SALDO INICIAL': 'vl_saldo_inicial'
}

dataset_saldo.rename(columns=rename_saldo, inplace=True)

In [4]:
dataset_saldo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ds_conta_bancaria  1 non-null      object        
 1   cd_agencia         1 non-null      int64         
 2   cd_conta_bancaria  1 non-null      int64         
 3   dt_saldo_inicial   1 non-null      datetime64[ns]
 4   vl_saldo_inicial   1 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 172.0+ bytes


Sem tratamento pois vem com a tipagem conforme mapa de aderencia do banco, adicionar validação?

In [5]:
dataset_saldo.head()

,ds_conta_bancaria,cd_agencia,cd_conta_bancaria,dt_saldo_inicial,vl_saldo_inicial
0,GERAL,123,78956,2023-01-01,2238863.82


## Tratamento CDC

In [6]:
rename_cdc = {
    'ds_normal_centro_custo': 'ds_normal_centro_custos',
    'sk_normal_centro_custo': 'sk_normal_centro_custos',
    'ds_centro_custo': 'ds_centro_custo'
}
dataset_cdc.rename(columns=rename_cdc, inplace=True)

# Organizando df
dataset_cdc = dataset_cdc.sort_values('sk_normal_centro_custos')

Vou dropar o index para refazer

In [7]:
dataset_cdc.head()

,ds_centro_custo,sk_normal_centro_custos,ds_normal_centro_custos
1,PRONTO SOCORRO,1,Pronto Atendimento
0,CENTRO CIRÚRGICO,2,Centro Cirurgico
2,CONSULTÓRIOS,3,Ambulatorio
3,UI POSTO 1,4,UI
4,UI POSTO 2,4,UI


In [8]:
dataset_cdc['cd_centro_custos'] = range(1, len(dataset_cdc)+1)

In [9]:
dataset_cdc.head()

,ds_centro_custo,sk_normal_centro_custos,ds_normal_centro_custos,cd_centro_custos
1,PRONTO SOCORRO,1,Pronto Atendimento,1
0,CENTRO CIRÚRGICO,2,Centro Cirurgico,2
2,CONSULTÓRIOS,3,Ambulatorio,3
3,UI POSTO 1,4,UI,4
4,UI POSTO 2,4,UI,5


In [10]:
dataset_cdc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 1 to 17
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ds_centro_custo          18 non-null     object
 1   sk_normal_centro_custos  18 non-null     int64 
 2   ds_normal_centro_custos  18 non-null     object
 3   cd_centro_custos         18 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 720.0+ bytes


Não é necessário fazer tipagem de dados devido aos dados já terem sido lidos na tipagem correta, adicionar validação?

## Tratamento Contas Financeiras

Reordenado para criação de código unico

In [11]:
dataset_contas.sort_values('sk_normal_conta_financeira', inplace=True)

In [12]:
dataset_contas.head()

,ds_conta_financeira,sk_normal_conta_financeira,ds_normal_conta_financeira
0,SALDO INICIAL,1,SALDO INICIAL
1,TRANFERÊNCIA ENTRE CONTAS (ENTRADA),2,TRANSFERÊNCIA ENTRE CONTAS
2,TRANFERÊNCIA ENTRE CONTAS (SAÍDA),2,TRANSFERÊNCIA ENTRE CONTAS
3,PROCEDIMENTOS CIRÚRGICOS (ENTRADA),3,RECEITAS OPERACIONAIS
4,RECEITAS CONVENIOS (RECEITAS OPERACIONAIS),3,RECEITAS OPERACIONAIS


In [13]:
dataset_contas['cd_conta_financeira'] = range(1, len(dataset_contas)+1)

In [14]:
dataset_contas.head(10)

,ds_conta_financeira,sk_normal_conta_financeira,ds_normal_conta_financeira,cd_conta_financeira
0,SALDO INICIAL,1,SALDO INICIAL,1
1,TRANFERÊNCIA ENTRE CONTAS (ENTRADA),2,TRANSFERÊNCIA ENTRE CONTAS,2
2,TRANFERÊNCIA ENTRE CONTAS (SAÍDA),2,TRANSFERÊNCIA ENTRE CONTAS,3
3,PROCEDIMENTOS CIRÚRGICOS (ENTRADA),3,RECEITAS OPERACIONAIS,4
4,RECEITAS CONVENIOS (RECEITAS OPERACIONAIS),3,RECEITAS OPERACIONAIS,5
5,UNIMED (RECEITAS OPERACIONAIS),3,RECEITAS OPERACIONAIS,6
6,BRADESCO SAÚDE (RECEITAS OPERACIONAIS),3,RECEITAS OPERACIONAIS,7
7,POSTAL SAUDE (RECEITAS OPERACIONAIS),3,RECEITAS OPERACIONAIS,8
8,RENDIMENTOS DE APLICACOES FINANCEIRAS (RECEITA...,4,RECEITAS NÃO OPERACIONAIS,9
9,RECEITAS DE ALUGUEL (RECEITAS NÃO OPERACIONAIS),4,RECEITAS NÃO OPERACIONAIS,10


In [15]:
teste_ = dataset_contas[((dataset_contas['ds_normal_conta_financeira'].str.contains('RECEITA', case=False)) & (dataset_contas['sk_normal_conta_financeira'] != 5)) | 
               ((dataset_contas['sk_normal_conta_financeira'].isin([1, 2])) & (dataset_contas['ds_conta_financeira'].str.contains('ENTRADA') | (dataset_contas['ds_conta_financeira'].str.contains('SALDO'))))]

Criando a coluna de débito e cédito

In [16]:

# Definindo a condição
condicao = (
    (dataset_contas['ds_normal_conta_financeira'].str.contains('RECEITA', case=False)) & (dataset_contas['sk_normal_conta_financeira'] != 5) |
    (dataset_contas['sk_normal_conta_financeira'].isin([1, 2])) & (dataset_contas['ds_conta_financeira'].str.contains('ENTRADA') | dataset_contas['ds_conta_financeira'].str.contains('SALDO'))
)

# Criando a coluna debito e crédito
dataset_contas['ds_unidade'] = np.where(condicao, 'C', 'D')


In [17]:
dataset_contas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 0 to 115
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ds_conta_financeira         116 non-null    object
 1   sk_normal_conta_financeira  116 non-null    int64 
 2   ds_normal_conta_financeira  116 non-null    object
 3   cd_conta_financeira         116 non-null    int64 
 4   ds_unidade                  116 non-null    object
dtypes: int64(2), object(3)
memory usage: 5.4+ KB


Os dados estão na tipagem requerida, adicionar validação?

## Tratamento fornecedores

In [18]:
dataset_fornecedor = dataset_lancamento['FORNECEDOR']
dataset_fornecedor = dataset_fornecedor.to_frame().reset_index()
dataset_fornecedor['FORNECEDOR'] = dataset_fornecedor['FORNECEDOR'].drop_duplicates()
dataset_fornecedor.dropna(inplace=True)
dataset_fornecedor = dataset_fornecedor.drop('index', axis=1)
dataset_fornecedor['sk_fornecedor'] = range(1, len(dataset_fornecedor)+1)
dataset_fornecedor['cd_fornecedor'] = dataset_fornecedor['sk_fornecedor']
dataset_fornecedor.rename(columns={'FORNECEDOR': 'ds_fornecedor'}, inplace=True)
dataset_fornecedor.head()

,ds_fornecedor,sk_fornecedor,cd_fornecedor
0,FORNECEDOR 444,1,1
1,FORNECEDOR 183,2,2
7,FORNECEDOR 416,3,3
8,FORNECEDOR 27,4,4
22,FORNECEDOR 917,5,5


## Tratamento lançamento

In [19]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [20]:
rename_lancamento = {
    'CONTA': 'ds_conta_financeira',
    'ID': 'pk_lancamento',
    'DATA VENCIMENTO': 'dt_vencimento',
    'MÊS REFERÊNCIA': 'dt_mes_referencia',
    'VALOR TOTAL': 'vl_total_nf',
    'VALOR PARCELA': 'vl_parcela',
    'FORNECEDOR': 'ds_fornecedor',
    'DATA PAGAMENTO': 'dt_realizado',
    'VALOR REALIZADO': 'vl_realizado',
    'CONSIDERAÇÕES LANÇAMENTO': 'ds_observacao',
    'NOTA FISCAL': 'cd_nf_doc',
    'STATUS': 'tp_situacao',
    'CENTRO DE CUSTO': 'ds_normal_centro_custos',
    'FORMA DE PAGAMENTO': 'tp_forma_pgto',
    'CONTA BANCARIA': 'ds_conta_bancaria'
}
dataset_lancamento.rename(columns=rename_lancamento, inplace=True)

# Criando coluna cd_lancamento
dataset_lancamento['cd_lancamento'] = dataset_lancamento['pk_lancamento']

Mesclando com left com o dataset de contas

In [21]:
dataset_lancamento = dataset_lancamento.merge(dataset_contas, how='left', on='ds_conta_financeira')

Criando a coluna sk_unidade

In [22]:
dataset_lancamento['sk_unidade'] = pd.factorize(dataset_lancamento['ds_unidade'])[0] +1

In [23]:
dataset_lancamento['sk_unidade'] = dataset_lancamento['sk_unidade'].replace(0, np.nan)

Criando coluna sk_conta_financeira

In [24]:
dataset_lancamento['sk_conta_financeira'] = dataset_lancamento['cd_conta_financeira']

Dropando as linhas completamente nulas

In [25]:
dataset_lancamento= dataset_lancamento.dropna(how='all')


Unindo com fornecedores

In [26]:
dataset_fornecedor.head(2)

,ds_fornecedor,sk_fornecedor,cd_fornecedor
0,FORNECEDOR 444,1,1
1,FORNECEDOR 183,2,2


In [27]:
dataset_lancamento = dataset_lancamento.merge(dataset_fornecedor, how='left', on='ds_fornecedor')

Criando coluna de data de vencimento da parcela 

In [28]:
dataset_lancamento['dt_venc_parcela'] = dataset_lancamento['dt_vencimento']

Criando coluna de data de emissao da nota e data de inclusao

In [29]:
dataset_lancamento['dt_emissao_nf_doc'] = ''

In [30]:
dataset_lancamento['dt_inclusao'] = ''

In [31]:
dataset_saldo['sk_conta_bancaria'] = pd.factorize(dataset_saldo['cd_conta_bancaria'])[0] +1
dataset_saldo['sk_conta_bancaria'] = dataset_saldo['sk_conta_bancaria'].replace(0, np.nan)

In [32]:
dataset_lancamento.head()

,pk_lancamento,ds_conta_financeira,ds_normal_centro_custos,dt_mes_referencia,dt_vencimento,ds_fornecedor,cd_nf_doc,vl_total_nf,vl_parcela,ds_observacao,tp_situacao,dt_realizado,vl_realizado,ds_conta_bancaria,tp_forma_pgto,cd_lancamento,sk_normal_conta_financeira,ds_normal_conta_financeira,cd_conta_financeira,ds_unidade,sk_unidade,sk_conta_financeira,sk_fornecedor,cd_fornecedor,dt_venc_parcela,dt_emissao_nf_doc,dt_inclusao
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,Farmacia,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN,6.0,11.0,CONSUMO DE MATERIAIS GERAIS,47.0,D,1.0,47.0,1,1,2023-04-28,,
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Tecnologia Informacao,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN,7.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-03-10,,
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN,8.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-03-27,,
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,UTI,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN,9.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-04-24,,
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN,10.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-05-22,,


In [33]:
dataset_lancamento['cd_parcela'] = 0
dataset_lancamento['vl_juros'] = 0.0
dataset_lancamento['vl_descontos'] = 0.0
dataset_lancamento['cd_documento'] = ''
dataset_lancamento['tp_lancamento'] = ''

Unindo com centro de custos

In [34]:
dataset_lancamento = dataset_lancamento.merge(dataset_cdc, how='left', on='ds_normal_centro_custos')

In [35]:
dataset_cdc.head()

,ds_centro_custo,sk_normal_centro_custos,ds_normal_centro_custos,cd_centro_custos
1,PRONTO SOCORRO,1,Pronto Atendimento,1
0,CENTRO CIRÚRGICO,2,Centro Cirurgico,2
2,CONSULTÓRIOS,3,Ambulatorio,3
3,UI POSTO 1,4,UI,4
4,UI POSTO 2,4,UI,5


In [36]:
dataset_lancamento.head()

,pk_lancamento,ds_conta_financeira,ds_normal_centro_custos,dt_mes_referencia,dt_vencimento,ds_fornecedor,cd_nf_doc,vl_total_nf,vl_parcela,ds_observacao,tp_situacao,dt_realizado,vl_realizado,ds_conta_bancaria,tp_forma_pgto,cd_lancamento,sk_normal_conta_financeira,ds_normal_conta_financeira,cd_conta_financeira,ds_unidade,sk_unidade,sk_conta_financeira,sk_fornecedor,cd_fornecedor,dt_venc_parcela,dt_emissao_nf_doc,dt_inclusao,cd_parcela,vl_juros,vl_descontos,cd_documento,tp_lancamento,ds_centro_custo,sk_normal_centro_custos,cd_centro_custos
0,6.0,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,Farmacia,2023-03-01,2023-04-28,FORNECEDOR 444,18185,15.00,15.00,Materiais Descartaveis Cozinha,PAGO,2023-04-01,15.00,GERAL,NaN,6.0,11.0,CONSUMO DE MATERIAIS GERAIS,47.0,D,1.0,47.0,1,1,2023-04-28,,,0,0.0,0.0,,,FARMÁCIA,8,18
1,7.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Tecnologia Informacao,2023-03-01,2023-03-10,FORNECEDOR 183,136874,36.53,36.53,Material de manutencao predial,PAGO,2023-03-01,36.53,GERAL,NaN,7.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-03-10,,,0,0.0,0.0,,,TECNOLOGIA DA INFORMAÇÃO,7,17
2,8.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-03-27,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-03-01,328.58,GERAL,NaN,8.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-03-27,,,0,0.0,0.0,,,CONSULTÓRIOS,3,3
3,9.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,UTI,2023-03-01,2023-04-24,FORNECEDOR 183,137186,328.58,328.58,Material de manutencao predial,PAGO,2023-04-01,328.58,GERAL,NaN,9.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-04-24,,,0,0.0,0.0,,,TERAPIA INTENSIVA,5,6
4,10.0,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,Ambulatorio,2023-03-01,2023-05-22,FORNECEDOR 183,137186,328.58,328.58,Fechadura,PAGO,2023-05-01,328.58,GERAL,NaN,10.0,14.0,MANUTENCAO E ASSISTENCIA TECNICA,71.0,D,1.0,71.0,2,2,2023-05-22,,,0,0.0,0.0,,,CONSULTÓRIOS,3,3


Dataset finalizado

In [37]:
select_column = [
                 'sk_unidade', 
                 'pk_lancamento',
                 'cd_lancamento',
                 'sk_conta_financeira',
                 'dt_vencimento',
                 'dt_mes_referencia',
                 'vl_total_nf',
                 'dt_venc_parcela',
                 'sk_fornecedor',
                 'dt_realizado',
                 'vl_realizado',
                 'ds_observacao',
                 'dt_emissao_nf_doc',
                 'dt_inclusao',
                 'cd_nf_doc',
                #  'sk_conta_bancaria',
                 'cd_parcela',
                 'vl_juros',
                 'vl_descontos',
                 'tp_lancamento',
                 'sk_normal_centro_custos',
                 'tp_forma_pgto',
                 'cd_documento',
                 'tp_situacao'
]

dataset_final = dataset_lancamento[select_column]

Selecionando as colunas que não estão no mapeamento original

In [38]:
dataset_saldo.head()

,ds_conta_bancaria,cd_agencia,cd_conta_bancaria,dt_saldo_inicial,vl_saldo_inicial,sk_conta_bancaria
0,GERAL,123,78956,2023-01-01,2238863.82,1


In [39]:
dataset_colunas_fora = dataset_lancamento.columns.difference(select_column)
dataset_fora = dataset_lancamento[dataset_colunas_fora]

Unindo o dataset de conta bancaria

In [40]:
dataset_fora = dataset_fora.merge(dataset_saldo, how='left', on='ds_conta_bancaria')

In [41]:
dataset_fora.head(2)

,cd_centro_custos,cd_conta_financeira,cd_fornecedor,ds_centro_custo,ds_conta_bancaria,ds_conta_financeira,ds_fornecedor,ds_normal_centro_custos,ds_normal_conta_financeira,ds_unidade,sk_normal_conta_financeira,vl_parcela,cd_agencia,cd_conta_bancaria,dt_saldo_inicial,vl_saldo_inicial,sk_conta_bancaria
0,18,47.0,1,FARMÁCIA,GERAL,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,FORNECEDOR 444,Farmacia,CONSUMO DE MATERIAIS GERAIS,D,11.0,15.00,123.0,78956.0,2023-01-01,2238863.82,1.0
1,17,71.0,2,TECNOLOGIA DA INFORMAÇÃO,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Tecnologia Informacao,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,36.53,123.0,78956.0,2023-01-01,2238863.82,1.0


In [42]:
dataset_final = pd.concat([dataset_final, dataset_fora], axis=1)

In [43]:
dataset_final.head()

,sk_unidade,pk_lancamento,cd_lancamento,sk_conta_financeira,dt_vencimento,dt_mes_referencia,vl_total_nf,dt_venc_parcela,sk_fornecedor,dt_realizado,vl_realizado,ds_observacao,dt_emissao_nf_doc,dt_inclusao,cd_nf_doc,cd_parcela,vl_juros,vl_descontos,tp_lancamento,sk_normal_centro_custos,tp_forma_pgto,cd_documento,tp_situacao,cd_centro_custos,cd_conta_financeira,cd_fornecedor,ds_centro_custo,ds_conta_bancaria,ds_conta_financeira,ds_fornecedor,ds_normal_centro_custos,ds_normal_conta_financeira,ds_unidade,sk_normal_conta_financeira,vl_parcela,cd_agencia,cd_conta_bancaria,dt_saldo_inicial,vl_saldo_inicial,sk_conta_bancaria
0,1.0,6.0,6.0,47.0,2023-04-28,2023-03-01,15.00,2023-04-28,1,2023-04-01,15.00,Materiais Descartaveis Cozinha,,,18185,0,0.0,0.0,,8,NaN,,PAGO,18,47.0,1,FARMÁCIA,GERAL,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,FORNECEDOR 444,Farmacia,CONSUMO DE MATERIAIS GERAIS,D,11.0,15.00,123.0,78956.0,2023-01-01,2238863.82,1.0
1,1.0,7.0,7.0,71.0,2023-03-10,2023-03-01,36.53,2023-03-10,2,2023-03-01,36.53,Material de manutencao predial,,,136874,0,0.0,0.0,,7,NaN,,PAGO,17,71.0,2,TECNOLOGIA DA INFORMAÇÃO,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Tecnologia Informacao,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,36.53,123.0,78956.0,2023-01-01,2238863.82,1.0
2,1.0,8.0,8.0,71.0,2023-03-27,2023-03-01,328.58,2023-03-27,2,2023-03-01,328.58,Material de manutencao predial,,,137186,0,0.0,0.0,,3,NaN,,PAGO,3,71.0,2,CONSULTÓRIOS,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Ambulatorio,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0
3,1.0,9.0,9.0,71.0,2023-04-24,2023-03-01,328.58,2023-04-24,2,2023-04-01,328.58,Material de manutencao predial,,,137186,0,0.0,0.0,,5,NaN,,PAGO,6,71.0,2,TERAPIA INTENSIVA,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,UTI,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0
4,1.0,10.0,10.0,71.0,2023-05-22,2023-03-01,328.58,2023-05-22,2,2023-05-01,328.58,Fechadura,,,137186,0,0.0,0.0,,3,NaN,,PAGO,3,71.0,2,CONSULTÓRIOS,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Ambulatorio,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0


## Definindo a tipagem dos dados

In [44]:
dataset_final.head()

,sk_unidade,pk_lancamento,cd_lancamento,sk_conta_financeira,dt_vencimento,dt_mes_referencia,vl_total_nf,dt_venc_parcela,sk_fornecedor,dt_realizado,vl_realizado,ds_observacao,dt_emissao_nf_doc,dt_inclusao,cd_nf_doc,cd_parcela,vl_juros,vl_descontos,tp_lancamento,sk_normal_centro_custos,tp_forma_pgto,cd_documento,tp_situacao,cd_centro_custos,cd_conta_financeira,cd_fornecedor,ds_centro_custo,ds_conta_bancaria,ds_conta_financeira,ds_fornecedor,ds_normal_centro_custos,ds_normal_conta_financeira,ds_unidade,sk_normal_conta_financeira,vl_parcela,cd_agencia,cd_conta_bancaria,dt_saldo_inicial,vl_saldo_inicial,sk_conta_bancaria
0,1.0,6.0,6.0,47.0,2023-04-28,2023-03-01,15.00,2023-04-28,1,2023-04-01,15.00,Materiais Descartaveis Cozinha,,,18185,0,0.0,0.0,,8,NaN,,PAGO,18,47.0,1,FARMÁCIA,GERAL,MATERIAL DESCARTAVEL (CONSUMO DE MATERIAIS GER...,FORNECEDOR 444,Farmacia,CONSUMO DE MATERIAIS GERAIS,D,11.0,15.00,123.0,78956.0,2023-01-01,2238863.82,1.0
1,1.0,7.0,7.0,71.0,2023-03-10,2023-03-01,36.53,2023-03-10,2,2023-03-01,36.53,Material de manutencao predial,,,136874,0,0.0,0.0,,7,NaN,,PAGO,17,71.0,2,TECNOLOGIA DA INFORMAÇÃO,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Tecnologia Informacao,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,36.53,123.0,78956.0,2023-01-01,2238863.82,1.0
2,1.0,8.0,8.0,71.0,2023-03-27,2023-03-01,328.58,2023-03-27,2,2023-03-01,328.58,Material de manutencao predial,,,137186,0,0.0,0.0,,3,NaN,,PAGO,3,71.0,2,CONSULTÓRIOS,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Ambulatorio,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0
3,1.0,9.0,9.0,71.0,2023-04-24,2023-03-01,328.58,2023-04-24,2,2023-04-01,328.58,Material de manutencao predial,,,137186,0,0.0,0.0,,5,NaN,,PAGO,6,71.0,2,TERAPIA INTENSIVA,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,UTI,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0
4,1.0,10.0,10.0,71.0,2023-05-22,2023-03-01,328.58,2023-05-22,2,2023-05-01,328.58,Fechadura,,,137186,0,0.0,0.0,,3,NaN,,PAGO,3,71.0,2,CONSULTÓRIOS,GERAL,PECAS E MATERIAIS DE MANUTENCAO - PREDIAL (MAN...,FORNECEDOR 183,Ambulatorio,MANUTENCAO E ASSISTENCIA TECNICA,D,14.0,328.58,123.0,78956.0,2023-01-01,2238863.82,1.0


In [45]:
dataset_final.columns

Index(['sk_unidade', 'pk_lancamento', 'cd_lancamento', 'sk_conta_financeira', 'dt_vencimento', 'dt_mes_referencia', 'vl_total_nf', 'dt_venc_parcela', 'sk_fornecedor', 'dt_realizado', 'vl_realizado', 'ds_observacao', 'dt_emissao_nf_doc', 'dt_inclusao', 'cd_nf_doc', 'cd_parcela', 'vl_juros', 'vl_descontos', 'tp_lancamento', 'sk_normal_centro_custos', 'tp_forma_pgto', 'cd_documento', 'tp_situacao', 'cd_centro_custos', 'cd_conta_financeira', 'cd_fornecedor', 'ds_centro_custo', 'ds_conta_bancaria', 'ds_conta_financeira', 'ds_fornecedor', 'ds_normal_centro_custos', 'ds_normal_conta_financeira', 'ds_unidade', 'sk_normal_conta_financeira', 'vl_parcela', 'cd_agencia', 'cd_conta_bancaria', 'dt_saldo_inicial', 'vl_saldo_inicial', 'sk_conta_bancaria'], dtype='object')

In [46]:
dataset_final.describe

<bound method NDFrame.describe of       sk_unidade  pk_lancamento  cd_lancamento  sk_conta_financeira dt_vencimento dt_mes_referencia  vl_total_nf dt_venc_parcela  sk_fornecedor dt_realizado  vl_realizado                   ds_observacao dt_emissao_nf_doc dt_inclusao cd_nf_doc  cd_parcela  vl_juros  vl_descontos tp_lancamento  sk_normal_centro_custos  tp_forma_pgto cd_documento tp_situacao  cd_centro_custos  cd_conta_financeira  cd_fornecedor           ds_centro_custo ds_conta_bancaria                                ds_conta_financeira   ds_fornecedor ds_normal_centro_custos             ds_normal_conta_financeira ds_unidade  sk_normal_conta_financeira  vl_parcela  cd_agencia  cd_conta_bancaria dt_saldo_inicial  vl_saldo_inicial  sk_conta_bancaria
0            1.0            6.0            6.0                 47.0    2023-04-28        2023-03-01        15.00      2023-04-28              1   2023-04-01         15.00  Materiais Descartaveis Cozinha                                   18185  

In [50]:
columns_type = {
    'sk_unidade': int, 
    'pk_lancamento': int,
    'cd_lancamento': int, 
    'sk_conta_financeira': int,
    'dt_vencimento': 'datetime64[ns]',
    'dt_mes_referencia': 'datetime64[ns]',
    'vl_total_nf': float, 
    'dt_venc_parcela': 'datetime64[ns]', 
    'sk_fornecedor': int, 
    'dt_realizado': 'datetime64[ns]', 
    'vl_realizado': float, 
    'ds_observacao': object, 
    'dt_emissao_nf_doc': 'datetime64[ns]', 
    'dt_inclusao': 'datetime64[ns]', 
    'cd_nf_doc': object, 
    # 'sk_conta_bancaria': int, 
    'cd_parcela': int, 
    'vl_juros': float, 
    'vl_descontos': float, 
    'tp_lancamento': object, 
    'sk_normal_centro_custos': int, 
    'tp_forma_pgto': object, 
    'cd_documento': object, 
    'tp_situacao': object
}

dataset_final = dataset_final.astype(columns_type)

In [53]:
dataset_final['cd_nf_doc'] = dataset_final['cd_nf_doc'].apply(lambda x: x.replace(' ', '').replace('-', '').replace('_', '').replace(';', '').replace('.', '').replace(',', '').zfill(15) if isinstance(x, str) else x)
dataset_final['cd_nf_doc'] = dataset_final['cd_nf_doc'].apply(lambda x: str(x)[-10:].rjust(10, '0'))

In [54]:
dataset_final.describe

<bound method NDFrame.describe of       sk_unidade  pk_lancamento  cd_lancamento  sk_conta_financeira dt_vencimento dt_mes_referencia  vl_total_nf dt_venc_parcela  sk_fornecedor dt_realizado  vl_realizado                   ds_observacao dt_emissao_nf_doc dt_inclusao   cd_nf_doc  cd_parcela  vl_juros  vl_descontos tp_lancamento  sk_normal_centro_custos tp_forma_pgto cd_documento tp_situacao  cd_centro_custos  cd_conta_financeira  cd_fornecedor           ds_centro_custo ds_conta_bancaria                                ds_conta_financeira   ds_fornecedor ds_normal_centro_custos             ds_normal_conta_financeira ds_unidade  sk_normal_conta_financeira  vl_parcela  cd_agencia  cd_conta_bancaria dt_saldo_inicial  vl_saldo_inicial  sk_conta_bancaria
0              1              6              6                   47    2023-04-28        2023-03-01        15.00      2023-04-28              1   2023-04-01         15.00  Materiais Descartaveis Cozinha               NaT         NaT  000001818

# Exportando para camada silver

In [137]:
output_folder = BASEFOLDER.format(stage='silver', folder='dados_recebidos')
today = datetime.now()
date = today.strftime('%Y_%m_%d')
file_name = f'dados_recebidos_extractDate={date}.csv'

output_path = os.path.join(output_folder, file_name)

In [138]:
if  not os.path.isdir(output_folder): 
    Path(output_folder).mkdir(parents=True, exist_ok=True)

In [139]:
dataset_final.to_csv(output_path, sep='\t')